In [ ]:
# Niestandardowy Callback dla Zaawansowanego Logowania:

import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Krok 1: Konfiguracja środowiska
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalizacja wartości pikseli do zakresu od 0 do 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Utworzenie zbioru danych z partiami do trenowania
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# Krok 2: Definicja modelu

model = Sequential([
    Flatten(input_shape=(28, 28)),  # Spłaszcz wejście do wektora 1D
    Dense(128, activation='relu'),  # Pierwsza warstwa ukryta z 128 neuronami i aktywacją ReLU
    Dense(10)  # Warstwa wyjściowa z 10 neuronami dla 10 klas (cyfry 0-9)
])

# Krok 3: Definicja funkcji straty, optymalizatora i metryki

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Funkcja straty dla klasyfikacji wieloklasowej
optimizer = tf.keras.optimizers.Adam()  # Optymalizator Adam dla efektywnego trenowania
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metryka do monitorowania dokładności podczas trenowania

from tensorflow.keras.callbacks import Callback

# Krok 4: Implementacja niestandardowego callbacku 
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'Koniec epoki {epoch + 1}, strata: {logs.get("loss")}, dokładność: {logs.get("accuracy")}')

# Krok 5: Implementacja pętli trenowania z niestandardowym callbackiem

epochs = 2
custom_callback = CustomCallback()  # Inicjalizacja niestandardowego callbacku

for epoch in range(epochs):
    print(f'Początek epoki {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Propagacja w przód: Obliczenie predykcji
            logits = model(x_batch_train, training=True)
            # Obliczenie straty
            loss_value = loss_fn(y_batch_train, logits)
        
        # Obliczenie gradientów
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Zastosowanie gradientów do aktualizacji wag modelu
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Aktualizacja metryki dokładności
        accuracy_metric.update_state(y_batch_train, logits)

        # Logowanie straty i dokładności co 200 kroków
        if step % 200 == 0:
            print(f'Epoka {epoch + 1} Krok {step}: Strata = {loss_value.numpy()} Dokładność = {accuracy_metric.result().numpy()}')
    
    # Wywołanie niestandardowego callbacku na końcu każdej epoki
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})
    
    # Resetowanie metryki na końcu każdej epoki
    accuracy_metric.reset_state()  # Użyj reset_state() zamiast reset_states()


In [ ]:
# Dodawanie metryki dokładności

import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Krok 1: Konfiguracja środowiska
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalizacja wartości pikseli do zakresu od 0 do 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Utworzenie zbioru danych z partiami do trenowania
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# Krok 2: Definicja modelu

model = Sequential([ 
    Flatten(input_shape=(28, 28)),  # Spłaszczenie wejścia do wektora 1D
    Dense(128, activation='relu'),  # Pierwsza warstwa ukryta z 128 neuronami i aktywacją ReLU
    Dense(10)  # Warstwa wyjściowa z 10 neuronami dla 10 klas (cyfry 0-9)
])

# Krok 3: Definicja funkcji straty, optymalizatora i metryki

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Funkcja straty dla klasyfikacji wieloklasowej
optimizer = tf.keras.optimizers.Adam()  # Optymalizator Adam dla efektywnego trenowania
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metryka do śledzenia dokładności podczas trenowania

# Krok 4: Implementacja pętli trenowania z metryką dokładności

epochs = 5  # Liczba epok trenowania

for epoch in range(epochs):
    print(f'Początek epoki {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Propagacja w przód: Obliczenie predykcji
            logits = model(x_batch_train, training=True)
            # Obliczenie straty
            loss_value = loss_fn(y_batch_train, logits)
        
        # Obliczenie gradientów
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Aktualizacja wag modelu poprzez zastosowanie gradientów
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Aktualizacja metryki dokładności
        accuracy_metric.update_state(y_batch_train, logits)

        # Logowanie straty i dokładności co 200 kroków
        if step % 200 == 0:
            print(f'Epoka {epoch + 1} Krok {step}: Strata = {loss_value.numpy()} Dokładność = {accuracy_metric.result().numpy()}')
    
    # Resetowanie metryki na końcu każdej epoki
    accuracy_metric.reset_state()


In [ ]:
# Podstawowa niestandardowa pętla trenowania

import os
import warnings
import tensorflow as tf 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

# Wyłączanie wszystkich ostrzeżeń Pythona
warnings.filterwarnings('ignore')

# Ustawienie poziomu logowania TensorFlow, aby wyciszyć ostrzeżenia i komunikaty informacyjne
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Krok 1: Konfiguracja środowiska
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# Krok 2: Definicja modelu

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

# Krok 3: Definicja funkcji straty i optymalizatora

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam()

# Krok 4: Implementacja niestandardowej pętli trenowania

epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Początek epoki {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Propagacja w przód
            loss_value = loss_fn(y_batch_train, logits)  # Obliczenie straty

        # Obliczenie gradientów i aktualizacja wag
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logowanie straty co 200 kroków
        if step % 200 == 0:
            print(f'Epoka {epoch + 1} Krok {step}: Strata = {loss_value.numpy()}')


2025-03-25 10:52:28.166021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742896348.329389    2594 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742896348.368198    2594 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742896348.667158    2594 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742896348.667202    2594 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742896348.667206    2594 computation_placer.cc:177] computation placer alr

Start of epoch 1
Epoch 1 Step 0: Loss = 2.432652711868286
Epoch 1 Step 200: Loss = 0.36350858211517334
Epoch 1 Step 400: Loss = 0.18423596024513245


KeyboardInterrupt: 